In [ ]:
!gdown --id 1HV6zeS_pqNrDOfHr3A7UvJKnyzy5u4K1
!gdown --id 1S6Eg67vDSUSx6usqnAYgHCR2oEYeNvmI
!gdown --id 1oJ5BIiGHOhD3hgWpJN2ddzLWgzLl-x_k

Downloading...
From: https://drive.google.com/uc?id=1HV6zeS_pqNrDOfHr3A7UvJKnyzy5u4K1
To: /content/Mal_sentiment_full_dev.tsv
100% 269k/269k [00:00<00:00, 39.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1S6Eg67vDSUSx6usqnAYgHCR2oEYeNvmI
To: /content/Mal_sentiment_full_train.tsv
100% 2.38M/2.38M [00:00<00:00, 37.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oJ5BIiGHOhD3hgWpJN2ddzLWgzLl-x_k
To: /content/Mal_sentiment_full_test_withoutlabels.tsv
100% 288k/288k [00:00<00:00, 38.1MB/s]


In [ ]:
!nvidia-smi

Fri Oct 22 09:55:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
tf.__version__

'2.6.0'

In [ ]:
!pip install tensorflow_text==2.6.0

     |████████████████████████████████| 4.4 MB 2.7 MB/s 


In [ ]:
!pip install bert-for-tf2

     |████████████████████████████████| 41 kB 141 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=0f74907e4cc8490a435a4344698d2f2e3180ecda4f3deb8941299cfb9952e362
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=44397e8251aea48df9ce31926457c3e2f2102b8146ead0e72321a3fe92309307
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7912 sha256=286e3527bdcec97f68f3105ecc03c183d70a60729e84d311702f134f7e405483
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params


In [ ]:
import math
import pandas as pd
import re
import numpy as np
import itertools
import collections
from collections import defaultdict, Counter
import string
from sklearn.preprocessing import LabelEncoder


import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
from bert import bert_tokenization
import numpy as np
from scipy.spatial import distance

import seaborn as sns
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt 
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
ls

Mal_sentiment_full_dev.tsv                 Mal_sentiment_full_train.tsv
Mal_sentiment_full_test_withoutlabels.tsv  sample_data/


In [ ]:
#train_df = pd.read_csv("Mal_sentiment_full_train.tsv", header=None, sep='\t')
train_df = pd.read_csv("Mal_sentiment_full_train.tsv", sep='\t')
print(len(train_df))

#test_df = pd.read_csv("Mal_sentiment_full_dev.tsv", header=None, sep='\t')
valid_df = pd.read_csv("Mal_sentiment_full_dev.tsv", sep='\t')
print(len(valid_df))

test_df = pd.read_csv("Mal_sentiment_full_test_withoutlabels.tsv", sep='\t')
len(test_df)

15888
1766


1962

In [ ]:
train_df

,text,category
0,Sunny chechi fans evide like adichu power kani...,unknown_state
1,Angane july month ile ende aadyathe leave njan...,Positive
2,ഏട്ടന്റെ പുതിയ പടത്തിനു വേണ്ടി കാത്തിരിക്കുന്ന...,unknown_state
3,ഇനി ലാലേട്ടന്റെ വേട്ട തുടങ്ങാൻ പോകുന്നു..........,Positive
4,Trailer powli oru raksha illa . Pakshea padam ...,Positive
...,...,...
15883,Minimum 10 thavana kandavar maathram like adi,unknown_state
15884,Please cinema kaannaaan kothi aakunnu onnu rel...,Positive
15885,USA ok.. India No... Sadhacharam,unknown_state
15886,nama hero december 12 wait & see mamamaga mah...,Positive


In [ ]:
train_df['category']

0         unknown_state
1              Positive
2         unknown_state
3              Positive
4              Positive
              ...      
15883     unknown_state
15884          Positive
15885     unknown_state
15886          Positive
15887    Mixed_feelings
Name: category, Length: 15888, dtype: object

In [ ]:
print(test_df['text'][279])
print(test_df['text'][516])
print(test_df['text'][546])
print(test_df['text'][555])
print(test_df['text'][559])
print(test_df['text'][570])
print(test_df['text'][825])
print(test_df['text'][838])
print(test_df['text'][860])
print(test_df['text'][890])
print(test_df['text'][1234])

Oru pani-palal manakunnu again Mr. JR :/
Ende ponno 68 vayasaya oraluda body thanneyano ith.... 🤩.. NAMICHU PONNIKKAAAA
FDFS.. trailer kandu ithu vareyum njanithu paranjittilla..🤩🤩
Degraders step BacK 🤫...  മമ്മൂക്ക
ഈ പാട്ട് ഡിസ്‌ലൈക് ചെയ്തവരുടെ വീട്ടുകാരെ സമ്മതിക്കണം .....BIG SALUTE
ഇത് ഏത് ഭാഷ ശിവനെ ഏത് ജില്ല 🤔🤔🤔🤔🤔🤔🤔
ഇതൊക്കെ മമ്മുക്ക പറഞ്ഞാൽ പൊളിക്കും 🤩 🤩 🤩
1:08 ST PIUS X KUTTIKANAM        *UYIR*🤩🤩🤩
Ente ponno..... Heavy.... BGM... Ikka pinne parayanilla....2 looks kidilosky 🤩🤩 Overall kidukkachi aahn Mone...
Song dislike adicha 90% perum phycho singles akum🤩
അഭ്യർത്ഥന ____________________ മാമാങ്കം പലകുറി കൊണ്ടാടും  എന്ന ഗാനം ടൈറ്റിൽ സോങ്ങ് ആയി ഉൾപ്പെടുത്താമോ....  Album :vasantha geethangal Singer: K J YESUDAS


In [ ]:
train_df['text'][0]

'Sunny chechi fans evide like adichu power kaniku🤣🤣'

In [ ]:
train_df

,text,category
0,Sunny chechi fans evide like adichu power kani...,unknown_state
1,Angane july month ile ende aadyathe leave njan...,Positive
2,ഏട്ടന്റെ പുതിയ പടത്തിനു വേണ്ടി കാത്തിരിക്കുന്ന...,unknown_state
3,ഇനി ലാലേട്ടന്റെ വേട്ട തുടങ്ങാൻ പോകുന്നു..........,Positive
4,Trailer powli oru raksha illa . Pakshea padam ...,Positive
...,...,...
15883,Minimum 10 thavana kandavar maathram like adi,unknown_state
15884,Please cinema kaannaaan kothi aakunnu onnu rel...,Positive
15885,USA ok.. India No... Sadhacharam,unknown_state
15886,nama hero december 12 wait & see mamamaga mah...,Positive


In [ ]:
valid_df

,text,category
0,Mammooka ninghal mass aa pwoli item,Positive
1,Waiting for Malayalam movie For Tamil paiyan,not-malayalam
2,ദളപതി ഫാൻസിന്റെ വക ഒരു ഒന്നൊന്നര വിജയാശംസകൾ...,Positive
3,#Trailer pwolichuuuu ഓണത്തിന് വന്നങ്ങു തകർത്തേ...,Positive
4,Mammoookkaaaa polichadukkiii katta waiting nv 21,Positive
...,...,...
1761,Aa ചിരി uff എന്റെ പൊന്നോ ഇക്ക vere ലെവൽ,Positive
1762,Katta katta katta katta waiting....cant wait man,Positive
1763,Arjun reddy bgm poole thonniyathu enniku mathr...,unknown_state
1764,Fahad ikka ithilum polikum en urappayi,Positive


In [ ]:
valid_df['text']

0                     Mammooka ninghal mass aa pwoli item
1           Waiting for Malayalam movie  For Tamil paiyan
2          ദളപതി ഫാൻസിന്റെ വക ഒരു ഒന്നൊന്നര വിജയാശംസകൾ...
3       #Trailer pwolichuuuu ഓണത്തിന് വന്നങ്ങു തകർത്തേ...
4        Mammoookkaaaa polichadukkiii katta waiting nv 21
                              ...                        
1761              Aa ചിരി uff എന്റെ പൊന്നോ ഇക്ക vere ലെവൽ
1762     Katta katta katta katta waiting....cant wait man
1763    Arjun reddy bgm poole thonniyathu enniku mathr...
1764               Fahad ikka ithilum polikum en urappayi
1765    Njan veendum kanan vannu 100 pravashyam enkilu...
Name: text, Length: 1766, dtype: object

In [ ]:
#removing all the emoji from the given data

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


for i in range(len(train_df)):
    temp1 = deEmojify(train_df['text'][i])
    train_df['text'][i] = temp1

for j in range(len(valid_df)):
    temp2 = deEmojify(valid_df['text'][j])
    valid_df['text'][j] = temp2

for k in range(len(test_df)):
    temp3 = deEmojify(test_df['text'][k])
    test_df['text'][k]=temp3

In [ ]:
print(test_df['text'][516])
print(test_df['text'][546])
print(test_df['text'][555])
print(test_df['text'][559])
print(test_df['text'][570])
print(test_df['text'][825])
print(test_df['text'][838])
print(test_df['text'][860])
print(test_df['text'][890])
print(test_df['text'][1234])

Ende ponno 68 vayasaya oraluda body thanneyano ith.... .. NAMICHU PONNIKKAAAA
FDFS.. trailer kandu ithu vareyum njanithu paranjittilla..
Degraders step BacK ...  മമ്മൂക്ക
ഈ പാട്ട് ഡിസ്‌ലൈക് ചെയ്തവരുടെ വീട്ടുകാരെ സമ്മതിക്കണം .....BIG SALUTE
ഇത് ഏത് ഭാഷ ശിവനെ ഏത് ജില്ല 
ഇതൊക്കെ മമ്മുക്ക പറഞ്ഞാൽ പൊളിക്കും   
1:08 ST PIUS X KUTTIKANAM        *UYIR*
Ente ponno..... Heavy.... BGM... Ikka pinne parayanilla....2 looks kidilosky  Overall kidukkachi aahn Mone...
Song dislike adicha 90% perum phycho singles akum
അഭ്യർത്ഥന ____________________ മാമാങ്കം പലകുറി കൊണ്ടാടും  എന്ന ഗാനം ടൈറ്റിൽ സോങ്ങ് ആയി ഉൾപ്പെടുത്താമോ....  Album :vasantha geethangal Singer: K J YESUDAS


In [ ]:
def preprocess_data(text):
  user_name = '@[A-Za-z0-9]+' # Pattern for matching the user names in the tweet
  has_tag = '#[A-Za-z0-9]+' # Pattern for matching the hasgtags in the tweer
  url = 'https?:\/\/[A-Za-z0-9\.\/\-]+' # Pattern for matching the URLs in the tweet
  #emoji_pattern = r'/[x{1F600}-x{1F64F}]/u'
  
  x = text.lower()  # Lower casing all the characters
  x = re.sub(user_name, '', x) # Replace the username with an empty characrer
  x = re.sub(has_tag,'',x) # Replace the hashtags with an empty characrer
  x = re.sub(url, '', x) # Replace the URLs with an empty characrer
  #x = re.sub(emoji_pattern, '', x)
  x = x.translate(str.maketrans('', '', string.punctuation)) # Replace all the characters except alphabets and digits from the tweet

  return x

In [ ]:
train_df['text'] = train_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
valid_df['text'] = valid_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
test_df['text'] = test_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe

In [ ]:
print(test_df['text'][279])
print(test_df['text'][516])
print(test_df['text'][546])
print(test_df['text'][555])
print(test_df['text'][559])
print(test_df['text'][570])
print(test_df['text'][825])
print(test_df['text'][838])
print(test_df['text'][860])
print(test_df['text'][890])
print(test_df['text'][1234])

oru panipalal manakunnu again mr jr 
ende ponno 68 vayasaya oraluda body thanneyano ith  namichu ponnikkaaaa
fdfs trailer kandu ithu vareyum njanithu paranjittilla
degraders step back   മമ്മൂക്ക
ഈ പാട്ട് ഡിസ്‌ലൈക് ചെയ്തവരുടെ വീട്ടുകാരെ സമ്മതിക്കണം big salute
ഇത് ഏത് ഭാഷ ശിവനെ ഏത് ജില്ല 
ഇതൊക്കെ മമ്മുക്ക പറഞ്ഞാൽ പൊളിക്കും   
108 st pius x kuttikanam        uyir
ente ponno heavy bgm ikka pinne parayanilla2 looks kidilosky  overall kidukkachi aahn mone
song dislike adicha 90 perum phycho singles akum
അഭ്യർത്ഥന  മാമാങ്കം പലകുറി കൊണ്ടാടും  എന്ന ഗാനം ടൈറ്റിൽ സോങ്ങ് ആയി ഉൾപ്പെടുത്താമോ  album vasantha geethangal singer k j yesudas


In [ ]:
valid_df['text'][0]

'mammooka ninghal mass aa pwoli item'

In [ ]:
train_text = train_df['text'].tolist()
valid_text = valid_df['text'].tolist()
test_text = test_df['text'].tolist()

train_label = train_df['category'].tolist()
valid_label = valid_df['category'].tolist()

In [ ]:
#function definition
def get_model(model_url, max_seq_length):
  inputs = dict(
    input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    ) 
  muril_layer = hub.KerasLayer(model_url, trainable=True)
  outputs = muril_layer(inputs)
  assert 'sequence_output' in outputs
  assert 'pooled_output' in outputs
  assert 'encoder_outputs' in outputs
  assert 'default' in outputs
  return tf.keras.Model(inputs=inputs,outputs=outputs["pooled_output"]), muril_layer
#function call
max_seq_length = 128
muril_model, muril_layer = get_model(
    model_url="https://tfhub.dev/google/MuRIL/1", max_seq_length=max_seq_length)

In [ ]:
#converts input into BERT-acceptable format (preprocessing)
vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
#That 3 types of embeddings in BERT
def create_input(input_strings, tokenizer, max_seq_length):
  input_ids_all, input_mask_all, input_type_ids_all = [], [], []
  for input_string in input_strings:
    input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    sequence_length = min(len(input_ids), max_seq_length)
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))
    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)
    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    input_type_ids_all.append([0] * max_seq_length)
  return np.array(input_ids_all), np.array(input_mask_all), np.array(input_type_ids_all)

In [ ]:
def encode(input_text):
  input_ids, input_mask, input_type_ids = create_input(input_text,tokenizer,max_seq_length)
  inputs = dict(
      input_word_ids=input_ids,
      input_mask=input_mask,
      input_type_ids=input_type_ids,
  )
  return muril_model(inputs)

In [ ]:
train

In [ ]:
import math
import time

In [ ]:
Batch_size = 64
start = 0;Endu = Batch_size
fullsize = math.floor(len(train_text)/Batch_size)

In [ ]:
train_text[0:10]

['sunny chechi fans evide like adichu power kaniku',
 'angane july month ile ende aadyathe leave njan angu edukuva july 5 nu',
 'ഏട്ടന്റെ പുതിയ പടത്തിനു വേണ്ടി കാത്തിരിക്കുന്നവർ ലൈക്\u200c ബട്ടൺ അടിച്ചു പോളിക്കോ',
 'ഇനി ലാലേട്ടന്റെ വേട്ട തുടങ്ങാൻ പോകുന്നു മുൻ കാല collection recordukal ഭേതിച്ചിരിക്കും',
 'trailer powli oru raksha illa  pakshea padam irangattea veruthea over degrade venda padam irangittu paraya',
 'malayalam padam thanne aanu 2020 vere level',
 'history of mammukka   kattawaiting and support love u mammukka',
 'yaaru andha mech le penne ',
 'ആമയെ വെള്ളത്തിൽ മുക്കി കൊല്ലുന്ന ഇനമാ ',
 'ജോഷി രണ്ടുംകല്പിച്ചു ആണെല്ലോ സംഭവം എന്തായാലും പൊളിക്കും']

In [ ]:
for i in range(fullsize):
    if i+1 == fullsize:
        Endu = len(train_text)
        out = encode(train_text[int(start):int(Endu)])
    else:
        out = encode(train_text[int(start):int(Endu)])
    if i == 0:
        total = out
    else:
        #torch.cat((total,out.last_hidden_state),0)
        total = tf.concat([total, out],0)
    start = Batch_size * (i+1)
    Endu = Batch_size * (i+2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247


In [ ]:
X_train = np.array(total)
np.save('Muril_X_train.npy', X_train)

In [ ]:
X_train = np.load('Muril_X_train.npy')

In [ ]:
X_train.shape

(15888, 768)

In [ ]:
Y_train = np.array(train_label)

In [ ]:
Batch_size = 64
start = 0;Endu = Batch_size
fullsize = math.floor(len(valid_text)/Batch_size)

In [ ]:
for i in range(fullsize):
    if i+1 == fullsize:
        Endu = len(valid_text)
        out = encode(valid_text[int(start):int(Endu)])
    else:
        out = encode(valid_text[int(start):int(Endu)])
    if i == 0:
        total = out
    else:
        #torch.cat((total,out.last_hidden_state),0)
        total = tf.concat([total, out],0)
    start = Batch_size * (i+1)
    Endu = Batch_size * (i+2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [ ]:
X_val = np.array(total)
np.save('Muril_X_val.npy', X_val)

In [ ]:
X_val = np.load('Muril_X_val.npy')

In [ ]:
Y_val = np.array(valid_label)

# SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
svc = SVC(decision_function_shape='ovo')
svc.fit(X_train,Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
pred_svc = svc.predict(X_val)

In [ ]:
confusion_matrix(Y_val,pred_svc)

array([[  0,   0, 102,   0,   0],
       [  0,   0, 237,   0,   0],
       [  0,   0, 704,   0,   2],
       [  0,   0, 141,   0,   0],
       [  0,   0, 580,   0,   0]])

In [ ]:
print(classification_report(Y_val,pred_svc))

                precision    recall  f1-score   support

Mixed_feelings       0.00      0.00      0.00       102
      Negative       0.00      0.00      0.00       237
      Positive       0.40      1.00      0.57       706
 not-malayalam       0.00      0.00      0.00       141
 unknown_state       0.00      0.00      0.00       580

      accuracy                           0.40      1766
     macro avg       0.08      0.20      0.11      1766
  weighted avg       0.16      0.40      0.23      1766



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
Dtree = DecisionTreeClassifier(criterion='entropy', random_state=10)

In [ ]:
Dtree.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=10, splitter='best')

In [ ]:
pred_Dtree = Dtree.predict(X_val)

In [ ]:
confusion_matrix(Y_val,pred_Dtree)

array([[ 17,  14,  44,   3,  24],
       [ 16,  61,  87,   8,  65],
       [ 39,  96, 352,  28, 191],
       [  1,   9,  34,  69,  28],
       [ 33,  82, 209,  18, 238]])

In [ ]:
print(classification_report(Y_val,pred_Dtree))

                precision    recall  f1-score   support

Mixed_feelings       0.16      0.17      0.16       102
      Negative       0.23      0.26      0.24       237
      Positive       0.48      0.50      0.49       706
 not-malayalam       0.55      0.49      0.52       141
 unknown_state       0.44      0.41      0.42       580

      accuracy                           0.42      1766
     macro avg       0.37      0.36      0.37      1766
  weighted avg       0.42      0.42      0.42      1766



## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC = RandomForestClassifier()
RFC.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
pred_RFC = RFC.predict(X_val)

In [ ]:
confusion_matrix(Y_val,pred_RFC)

array([[  7,   0,  69,   1,  25],
       [  1,  24, 146,   0,  66],
       [  1,   0, 585,   5, 115],
       [  0,   0,  58,  68,  15],
       [  1,   3, 274,   3, 299]])

In [ ]:
print(classification_report(Y_val,pred_RFC))

                precision    recall  f1-score   support

Mixed_feelings       0.70      0.07      0.13       102
      Negative       0.89      0.10      0.18       237
      Positive       0.52      0.83      0.64       706
 not-malayalam       0.88      0.48      0.62       141
 unknown_state       0.57      0.52      0.54       580

      accuracy                           0.56      1766
     macro avg       0.71      0.40      0.42      1766
  weighted avg       0.63      0.56      0.51      1766



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/Muril_X_train.npy" -r "/content/drive/MyDrive/MtechTesis/MuRIL"

In [ ]:
!cp "/content/Muril_X_val.npy" -r "/content/drive/MyDrive/MtechTesis/MuRIL"

In [ ]:
import cupy as cp

In [ ]:
X_train = cp.load('Muril_X_train.npy')
X_val = cp.load('Muril_X_val.npy')
Y_train = cp.load('Y_train.npy')
Y_val = cp.load('Y_val.npy')

In [ ]:
from WR_LS_Denoising import WR_LS_Denoise

In [ ]:
start = time.ctime(time.time())
print(start)
for i in range(len(X_train)):
    Denoised =  WR_LS_Denoise(X_train[i])
    X_train[i] = X_train[i]
    if(i%1000 == 0):
        print(i, time.ctime(time.time()))

Fri Oct 22 10:43:43 2021
0 Fri Oct 22 10:43:52 2021
1000 Fri Oct 22 10:44:32 2021
2000 Fri Oct 22 10:45:11 2021
3000 Fri Oct 22 10:45:50 2021
4000 Fri Oct 22 10:46:30 2021
5000 Fri Oct 22 10:47:09 2021
6000 Fri Oct 22 10:47:48 2021
7000 Fri Oct 22 10:48:28 2021
8000 Fri Oct 22 10:49:07 2021
9000 Fri Oct 22 10:49:46 2021
10000 Fri Oct 22 10:50:25 2021
11000 Fri Oct 22 10:51:04 2021
12000 Fri Oct 22 10:51:43 2021
13000 Fri Oct 22 10:52:23 2021
14000 Fri Oct 22 10:53:02 2021
15000 Fri Oct 22 10:53:41 2021


In [ ]:
start = time.ctime(time.time())
print(start)
for i in range(len(X_val)):
    Denoised =  WR_LS_Denoise(X_val[i])
    X_val[i] = X_val[i]
    if(i%1000 == 0):
        print(i, time.ctime(time.time()))

Fri Oct 22 10:56:11 2021
0 Fri Oct 22 10:56:11 2021
1000 Fri Oct 22 10:56:51 2021


In [ ]:
WR_LS_Denoised_X_train = cp.save('WR_LS_Denoised_MuRIL_X_train.npy',X_train)
WR_LS_Denoised_X_val = cp.save('WR_LS_Denoised_MuRIL_X_val.npy',X_val)

In [ ]:
X_train = np.load('WR_LS_Denoised_MuRIL_X_train.npy')
X_val = np.load('WR_LS_Denoised_MuRIL_X_val.npy')

In [ ]:
Y_train = np.load('Y_train.npy')
Y_val = np.load('Y_val.npy')

# SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
svc = SVC(decision_function_shape='ovo')
svc.fit(X_train,Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
pred_svc = svc.predict(X_val)

In [ ]:
confusion_matrix(Y_val,pred_svc)

array([[  0,   0, 102,   0,   0],
       [  0,   0, 237,   0,   0],
       [  0,   0, 704,   0,   2],
       [  0,   0, 141,   0,   0],
       [  0,   0, 580,   0,   0]])

In [ ]:
print(classification_report(Y_val,pred_svc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       102
           1       0.00      0.00      0.00       237
           2       0.40      1.00      0.57       706
           3       0.00      0.00      0.00       141
           4       0.00      0.00      0.00       580

    accuracy                           0.40      1766
   macro avg       0.08      0.20      0.11      1766
weighted avg       0.16      0.40      0.23      1766



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC = RandomForestClassifier()
RFC.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
pred_RFC = RFC.predict(X_val)

In [ ]:
print(classification_report(Y_val,pred_RFC))

              precision    recall  f1-score   support

           0       0.64      0.07      0.12       102
           1       0.82      0.11      0.20       237
           2       0.52      0.86      0.65       706
           3       0.87      0.47      0.61       141
           4       0.61      0.51      0.56       580

    accuracy                           0.57      1766
   macro avg       0.69      0.40      0.43      1766
weighted avg       0.63      0.57      0.53      1766



# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
Dtree = DecisionTreeClassifier(criterion='entropy', random_state=10)

In [ ]:
Dtree.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=10, splitter='best')

In [ ]:
pred_Dtree = Dtree.predict(X_val)

In [ ]:
confusion_matrix(Y_val,pred_Dtree)

array([[ 17,  14,  44,   3,  24],
       [ 16,  61,  87,   8,  65],
       [ 39,  96, 352,  28, 191],
       [  1,   9,  34,  69,  28],
       [ 33,  82, 209,  18, 238]])

In [ ]:
print(classification_report(Y_val,pred_Dtree))

              precision    recall  f1-score   support

           0       0.16      0.17      0.16       102
           1       0.23      0.26      0.24       237
           2       0.48      0.50      0.49       706
           3       0.55      0.49      0.52       141
           4       0.44      0.41      0.42       580

    accuracy                           0.42      1766
   macro avg       0.37      0.36      0.37      1766
weighted avg       0.42      0.42      0.42      1766

